In [1]:
import pandas as pd
import re
import random
# itertools — Functions creating iterators for efficient looping
# The module standardizes a core set of fast, memory efficient tools that are useful by themselves or in combination. 
from itertools import product,combinations

C:\Users\PRIME1\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


[itertools](https://docs.python.org/3/library/itertools.html)

In [198]:
file_path = 'test_data/corpus-q3.csv'
df = pd.read_csv(file_path)
#df

In [3]:
# Persons of verbs
# [1-3][SP] = [(eu,nós),(tu,vós),(ele,eles)]

nums = ['1','2','3']
lts = ['S','P']

combinations = product(nums,lts, repeat=1)
combinations = product(combinations, repeat=2)

# Toal combinations for directionality augmentation
mylist = []
for i in combinations:
    
    left = i[0][0]+i[0][1] #left side of the verb
    rigth = i[1][0]+i[1][1] #rigth side of the verb
    
    mylist.append((left,rigth))

In [43]:
def directionality_augmentation(tuple_string):
    '''Directionality augmentation of phrase gr, miss gi'''
    
    # Lembrar de não usar \w , substituir por: REGEX_LATIN = 'A-ZÁÉÍÓÚÀÂÊÔÃÕÜÇa-záéíóúàâêôãõüç'
    w ='[A-ZÁÉÍÓÚÀÂÊÔÃÕÜÇa-záéíóúàâêôãõüç]+'

    pattern1 = '[1-3][SP](_'+w+'_)[1-3][SP]'
    pattern2 = '[1-3][SP]_'+w+'_[1-3][SP]'

    x = re.findall(pattern1,tuple_string[1])

    left_verb = re.findall('(.+)'+pattern2,
                           tuple_string[1])#pegando o lado esquerdo do verbo

    right_verb = re.findall(pattern2+'(.+)',
                            tuple_string[1])#pegando o lado direito do verbo

    result_all = []
    for person in mylist:
        verb = person[0] + x[0] + person[1]

        result_all.append(left_verb[0] + verb[0] + right_verb[0])
        
    return result_all


In [197]:
tuple_string = (df['gr'][0],df['gi'][0])

#directionality_augmentation(tuple_string)

<h2>TODO</h2>

In [180]:
# Possíveis variáveis de classe
agents_dict = {

    '1S': ['EU'],
    '2S': ['TU'], 
    '3S': ['ELE', 'ELA'],
    '1P': ['NÓS'],
    '2P': ['VÓS'],
    '3P': ['ELES', 'ELAS']

}



pronouns_dict = {

    '1S': ['ME'],
    '2S': ['TE'],
    '3S': ['LHE'],
    '1P': ['NÓS'],
    '2P': ['VÓS'],
    '3P': ['LHES']

}
regex_latin ='[A-ZÁÉÍÓÚÀÂÊÔÃÕÜÇa-záéíóúàâêôãõüç]+'

# agentes
agent_pattern = r'\b(EU|TU|ELE|ELA|NÓS|VÓS|ELES|ELAS)\b'

# verbos
verb_pattern = r'\b({0}R)\b'.format(regex_latin)
# pronome obliquo atono
pronoun_pattern = r'\b(ME|TE|LHE|VOS|NOS|LHES)\b'

pattern_agent_verb = "%s %s %s" %(agent_pattern, verb_pattern, agent_pattern)
pattern_pronoun_verb = "%s %s %s" %(agent_pattern, pronoun_pattern, verb_pattern)

phrases = ['EU ME REMEXER MUITO', 'OI EU ABRAÇAR TU PASSARO TU TRABALHAR EU QUIMO', 'NOSSA EU ABRAÇÉRAAR ELE BALEIA', 'COALA CAPIM EU ABRAÇAR VÓS ANIMAL']

behind_ahead = r'(.+){0}(.+)'.format(pattern_agent_verb)
search_pattern_agent_verb = re.findall(pattern_agent_verb, phrases[1])
search_results = re.finditer(behind_ahead, phrases[1])

#print(search_pattern_agent_verb[0])

### Detectar os padrões das frases

In [181]:
def find_pattern(phrase):
    '''Find pattern in phrase, return 2 list with patterns found'''
   
    regex_latin ='[A-ZÁÉÍÓÚÀÂÊÔÃÕÜÇa-záéíóúàâêôãõüç]+'
    # agentes
    agent_pattern = r'\b(EU|TU|ELE|ELA|NÓS|VÓS|ELES|ELAS)\b'

    # verbos
    verb_pattern = r'\b({0}R)\b'.format(regex_latin)
    # pronome obliquo atono
    pronoun_pattern = r'\b(ME|TE|LHE|VOS|NOS|LHES)\b'
    
    
    pattern_agent_verb = "%s %s %s" %(agent_pattern, verb_pattern, agent_pattern)
    pattern_pronoun_verb = "%s %s %s" %(agent_pattern, pronoun_pattern, verb_pattern)

    # resultado do regex para o padrão [agente -> verbo -> receptor]
    search_pattern_agent_verb = re.findall(pattern_agent_verb, phrase)
  
    # resultado do regex para o padrão [agente -> pronome -> verbo]
    search_pattern_pronoun_verb = re.findall(pattern_pronoun_verb, phrase)
    
    return search_pattern_agent_verb, search_pattern_pronoun_verb

### Remontando o código

In [182]:
def for_string(pattern):
    '''Transformando a padrão em [agente -> verbo -> receptor] em string para pesquisar
            a parte de trás e da frente do padrão'''
    string = ''
    for index, part in enumerate(pattern):
        if not index:
            string +=part
        else:
            string += ' '+part
    return string

In [225]:

def new_patterns1(pattern):
    '''cria uma novas frases a partir de um pattern [agente -> verbo -> receptor]'''
    
    phrases = []
    
    verb = pattern[1]
    # combinação com todos as possibilidades do dicionário
    combinations = product(agents_dict.keys(), agents_dict.keys(), repeat=1)

    for item in combinations:
        # criação de frases do tipo [EU, TU ...]
        # os for's são necessários nos casos de [ELE, ELA, ELES ELAS]
        for index_1, item_1  in enumerate(agents_dict[item[0]]):

            for index_2, item_2 in enumerate(agents_dict[item[1]]):

                gr = f'{item_1} {verb} {item_2}'
                #gi = f'{item[0]}_{verb}_{item[1]}'
                phrases.append(gr)
  
    return phrases

def new_patterns2(pattern):
    '''cria uma novas frases a partir de um pattern [agente -> pronome -> verbo]'''
    phrases = []
    
    verb = pattern[2]
   
    # combinação com todos as possibilidades do dicionário pronouns_dict
    combinations = product(agents_dict.keys(), pronouns_dict.keys(), repeat=1)

    for item in combinations:
        # criação de frases do tipo [EU, TU ...]
        # os for's são necessários nos casos de [ELE, ELA, ELES ELAS]
        for index_1, item_1  in enumerate(agents_dict[item[0]]):

            for index_2, item_2 in enumerate(pronouns_dict[item[1]]):

                gr = f'{item_1} {item_2} {verb}'
                #gi = f'{item[0]}_{verb}_{item[1]}'
                phrases.append(gr)
                
    return phrases


In [255]:

def assembly_phrase1(phrase,search_pattern_agent_verb,item):
    '''Monta a frase dado uma combinção [agente -> verbo -> receptor]'''
    pstr = []
    for pattern in search_pattern_agent_verb:
        pstr.append(for_string(pattern))
    
    new_phrase = re.sub(pstr[0],item[0],phrase)
    
    for i in range(1,len(pstr)):
        
        new_phrase = re.sub(pstr[i],item[i],new_phrase)
    
    return new_phrase



def assembly_phrase2(phrase,search_pattern_pronoun_verb,item):
    '''Monta a frase dado uma combinção  [agente -> pronome -> verbo]'''
    pstr = []
    for pattern in search_pattern_pronoun_verb:
        pstr.append(for_string(pattern))
    
    new_phrase = re.sub(pstr[0],item[0],phrase)
    #print(new_phrase)
    
    for i in range(1,len(pstr)):
        
        new_phrase = re.sub(pstr[i],item[i],new_phrase)
    
    return new_phrase


## main

In [256]:
frase = ['25 AGOSTO SAIR COM REVISTA BLITZ COLETÂNEA ALMADA 79 QUE REVELAR PRIMEIRAS MAQUETA ROCK PORTUGUÊS REGISTAR 1979 AINDA TEMA CAÇADA GRAVAR CONVENTO CAPUCHOS 1995 JORGE MORRER CAPTAR VIVO CENTRO CULTURAL BELÉM 2013 [PONTO] ANO QUE PASSAR 30 ANO SOBRE MORTE JOSÉ AFONSO OS UHF LANÇAR 27 OUTUBRO HERANÇA ANDARILHO ÁLBUM TRIBUTO MAIOR REFERÊNCIA MÚSICA NACIONAL [PONTO] ESTE DISCO SER DAR CONTINUIDADE FANTÁSTICO OBRA QUE ELE NOS DEIXAR REVELAR SEMENTE HERDAR REVELAR ANTÓNIO 1000º [PONTO] RIBEIRO PRODUTOR PROJETO ACRECENTOU ESTE SER AQUELES DISCO QUE SE FAZER VEZ VIDA [PONTO] SER 7 VERSÃO PUXAR PARA SOM ORA ELÉCTRICO ORA ACÚSTICO QUE SE JUNTAR 3 TEMA GRUPO FIO CONDUTOR LEGADO HERANÇA MAIS IMPORTANTE CANTAUTOR PORTUGUÊS [PONTO] PRIMEIRA AMOSTRA IR TEMA TRAZER OUTRO AMIGO TAMBÉM REVELAR 24 ABRIL 2017 ESTREIA ANTENA 1 COMEMORAR 43 ANO REVOLUÇÃO ABRIL SEGUIR SINGLE COMBOIO DESCENDENTE COM PARTICIPAÇÃO ARMANDO TEIXEIRA [PONTO]']

In [284]:
phrases = ['EU TE CONHECER [PONTO]']#'NÓS CONFIAR ELE [PONTO]'
new_patterns = []
new_p = []
for phrase in phrases:

    # [agente -> verbo -> receptor] and [agente -> pronome -> verbo]
    search_pattern_agent_verb, search_pattern_pronoun_verb = find_pattern(phrase)
    
    # Shuffle nos pattern encontradas para para retirada de apenas 2 de forma aleatória,
    # porque para cada padrão é gerada 64 frases, logo é 64 ^ NumeroDePatterns
    random.shuffle(search_pattern_agent_verb)
    random.shuffle(search_pattern_agent_verb)
    
    search_pattern_agent_verb = search_pattern_agent_verb[:2]
    search_pattern_pronoun_verb = search_pattern_pronoun_verb[:2]
    
    print('Partes a serem modificadas:',search_pattern_agent_verb,search_pattern_pronoun_verb)
    
    #[agente -> verbo -> receptor] 
    if search_pattern_agent_verb:
        
        for patterns_in_phrase in search_pattern_agent_verb:
            s = new_patterns1(patterns_in_phrase)
            new_patterns.append(s)

        #combinação dos pattern das frases geradas
        combination_patterns= product(*new_patterns, repeat=1)

        for cont, item in enumerate(combination_patterns):
            new_p.append(assembly_phrase(phrase,search_pattern_agent_verb,item))
    
    #[agente -> pronome -> verbo]
    if search_pattern_pronoun_verb:
        
        for patterns_in_phrase in search_pattern_pronoun_verb:
            
            s = new_patterns2(patterns_in_phrase)
            new_patterns.append(s)
        #combinação dos pattern das frases geradas
        combination_patterns= product(*new_patterns, repeat=1)

        for cont, item in enumerate(combination_patterns):
            #print(search_pattern_pronoun_verb)
            new_p.append(assembly_phrase2(phrase,search_pattern_pronoun_verb,item))

            
    # Shuffle para retornar 50 frases aleatórias
    random.shuffle(new_p)

Partes a serem modificadas: [] [('EU', 'TE', 'CONHECER')]


In [287]:
new_p[:1]

['ELA LHE CONHECER [PONTO]']

In [286]:
len(new_p)

48

## Testando com dados reais

In [195]:
test_data = pd.read_csv('test_data/gr_gi.csv')

In [141]:
test_data.columns = ['gr','gi']

In [142]:
# Pegando as frases que possuem a necessidade de augmentation
# [agente -> verbo -> receptor] and [agente -> pronome -> verbo]
cont = 0
phrases_agent_verb = []
phrases_pronoun_verb = []
for i,phrase in enumerate(test_data['gr']):
    
    try:
        search_pattern_agent_verb, search_pattern_pronoun_verb = find_pattern(phrase)

        if search_pattern_agent_verb :
            tpl = (test_data['gr'][i],test_data['gi'][i])
            phrases_agent_verb.append(tpl)

            cont+=1
        if search_pattern_pronoun_verb:
            tpl = (test_data['gr'][i],test_data['gi'][i])
            phrases_pronoun_verb.append(tpl)
            cont +=1
        #if cont >= 10:
            #break
    except Exception as e:
        print(e)

expected string or bytes-like object


In [128]:
# Total de frases [agente -> verbo -> receptor]
len(phrases_agent_verb)

58

In [129]:
# Total de frases [agente -> pronome -> verbo]
len(phrases_pronoun_verb)

537

In [265]:
phrases_pronoun_verb[random.randint(0,len(phrases_pronoun_verb))]

('NÓS NOS IMPORTAR [PONTO]', 'NÓS IMPORTAR_IMPORTÂNCIA [PONTO]')

In [262]:
phrases_agent_verb[random.randint(0,len(phrases_agent_verb))]

('NÓS CONFIAR ELE [PONTO]', 'NÓS CONFIAR ELA [PONTO]')